In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# 順序ロジスティック回帰用ライブラリ (statsmodels)
from statsmodels.miscmodels.ordinal_model import OrderedModel
import japanize_matplotlib


# 入出力ファイルの設定
INPUT_FILE_NAME = 'data.csv'
OUTPUT_IMAGE_NAME = 'odds_ratios.png'

# フォント設定
plt.rcParams['font.size'] = 12

# カラム名の英語変換マッピング
rename_dict = {
    # 属性
    "年齢を教えてください。": "Age",
    "性別を教えてください。": "Gender",
    
    # 味覚項目
    "甘味をどの程度感じましたか？（１／５）": "Sweetness",
    "うま味をどの程度感じましたか？（２／５）": "Umami",
    "苦味をどの程度感じましたか？（３／５）": "Bitterness",
    "酸味をどの程度感じましたか？（４／５）": "Sourness",
    "塩味をどの程度感じましたか？（５／５）": "Saltiness",
    
    # 視覚項目
    "華やかさをどの程度感じましたか？（１／５）": "Floral",
    "開放感をどの程度感じましたか？（２／５）": "Openness",
    "爽やかさをどの程度感じましたか？（３／５）": "Freshness",
    "賑わい感をどの程度感じましたか？（４／５）": "Liveliness",
    "賑い感をどの程度感じましたか？（４／５）": "Liveliness", 
    "高揚感をどの程度感じましたか？（５／５）": "Excitement",
    
    # 目的変数
    "クラフトビール「白蒲田」をまた飲んでみたいと思いますか？": "Drink_Again"
}

# 目的変数
TARGET_VARIABLE = "Drink_Again"

# Lasso回帰で選択された変数のリスト
# 前のステップ(Phase2_Lasso)の結果を見て、ここに「選ばれた変数」を記入
SELECTED_VARIABLES = [
    "Floral",      # 華やかさ
    "Umami",       # うま味
    "Excitement",  # 高揚感
    "Gender_男性"  # 性別（ダミー変数化した後の名前）
]

# ==========================================
# メイン処理
# ==========================================

# データの読み込みと前処理
try:
    df = pd.read_csv(INPUT_FILE_NAME, encoding='shift_jis')
    print(f"データ読み込み成功: {INPUT_FILE_NAME}")
except FileNotFoundError:
    print(f"エラー: ファイル '{INPUT_FILE_NAME}' が見つかりません。")
    # ダミーデータ生成（動作確認用）
    df = pd.DataFrame(np.random.randint(1, 6, (50, 13)), columns=list(rename_dict.keys()))
    df['性別を教えてください。'] = np.random.choice(['男性', '女性'], 50)

# リネーム
df_renamed = df.rename(columns=rename_dict)

# 分析用データの作成
valid_cols = [col for col in rename_dict.values() if col in df_renamed.columns]
df_analysis = df_renamed[valid_cols]

# 数値変換（性別以外）
num_cols = [c for c in df_analysis.columns if c != 'Gender']
for col in num_cols:
    df_analysis[col] = pd.to_numeric(df_analysis[col], errors='coerce')

# 欠損除去
df_analysis = df_analysis.dropna()

# 性別のダミー変数化 (男性=1, 女性=0)
# drop_first=True で多重共線性を回避
df_analysis = pd.get_dummies(df_analysis, columns=['Gender'], drop_first=True)

print("データ前処理完了。")
print("使用する変数一覧:", df_analysis.columns.tolist())

# ==========================================
# 順序ロジスティック回帰の実行
# ==========================================

def fit_ordinal_model(data, target_col, feature_cols, model_name="Model"):
    """順序ロジスティック回帰モデルを構築して結果を表示する関数"""
    
    # 説明変数(X)と目的変数(y)
    X = data[feature_cols]
    y = data[target_col]
    
    # statsmodelsのOrderedModelを使用
    # distr='logit' でロジスティック分布（Proportional Odds Model）を指定
    try:
        model = OrderedModel(y, X, distr='logit')
        result = model.fit(method='bfgs', disp=False)
        
        print(f"\n{'='*20} {model_name} の結果 {'='*20}")
        print(result.summary())
        print(f"AIC: {result.aic:.2f}")
        return result
    except Exception as e:
        print(f"モデル構築エラー ({model_name}): {e}")
        return None

# パターンA: 全変数モデル
# 目的変数以外の全ての変数を使用
all_features = [c for c in df_analysis.columns if c != TARGET_VARIABLE]
result_full = fit_ordinal_model(df_analysis, TARGET_VARIABLE, all_features, "Full Model (全変数)")

# パターンB: 選択変数モデル
# Lassoで選ばれた変数のみを使用
# ※データ内に存在しない変数がリストにある場合は除外して実行
valid_selected_vars = [c for c in SELECTED_VARIABLES if c in df_analysis.columns]

if len(valid_selected_vars) > 0:
    result_selected = fit_ordinal_model(df_analysis, TARGET_VARIABLE, valid_selected_vars, "Selected Model (変数選択後)")
else:
    print("\n※ 指定された選択変数がデータ内に見つからないため、選択モデルの実行をスキップします。")
    result_selected = None


# ==========================================
# モデル比較とオッズ比の可視化
# ==========================================

best_result = None
best_model_name = ""

# AICによるモデル比較 (小さい方が良い)
if result_full is not None and result_selected is not None:
    print("\n【モデル比較 (AIC)】")
    print(f"Full Model AIC:     {result_full.aic:.2f}")
    print(f"Selected Model AIC: {result_selected.aic:.2f}")
    
    if result_selected.aic < result_full.aic:
        print(">> Selected Model (変数選択後) の方が当てはまりが良いです。")
        best_result = result_selected
        best_model_name = "Selected Model"
    else:
        print(">> Full Model (全変数) の方が当てはまりが良いです。")
        best_result = result_full
        best_model_name = "Full Model"
elif result_selected is not None:
    best_result = result_selected
    best_model_name = "Selected Model"
else:
    best_result = result_full
    best_model_name = "Full Model"

# オッズ比の計算とプロット
if best_result is not None:
    # 係数(params)と信頼区間(conf_int)を取得
    params = best_result.params
    conf = best_result.conf_int()
    
    # 閾値パラメータ（1/2, 2/3...）を除外し、説明変数の係数だけ抽出
    # OrderedModelのparamsには、変数の係数と、カットポイント(閾値)が含まれる
    # 変数の係数は最初の方にある
    num_vars = len(best_result.model.exog_names)
    variable_params = params[:num_vars]
    variable_conf = conf.iloc[:num_vars]
    
    # オッズ比に変換 (exp)
    odds_ratios = np.exp(variable_params)
    error_bars = [
        np.exp(variable_params) - np.exp(variable_conf[0]), # 下側誤差
        np.exp(variable_conf[1]) - np.exp(variable_params)  # 上側誤差
    ]
    
    # データフレーム化
    or_df = pd.DataFrame({
        'Variable': variable_params.index,
        'Odds Ratio': odds_ratios,
        'Lower CI': np.exp(variable_conf[0]),
        'Upper CI': np.exp(variable_conf[1])
    }).sort_values('Odds Ratio', ascending=False)
    
    print(f"\n【{best_model_name} のオッズ比 (Odds Ratios)】")
    print("※ 1.0より大きいと「リピート意向を高める要因」、小さいと「下げる要因」")
    display(or_df.round(3))
    
    # グラフ描画
    plt.figure(figsize=(10, 6))
    
    # 1.0のライン（基準線）
    plt.axvline(x=1, color='red', linestyle='--', linewidth=1, label="No Effect (OR=1)")
    
    # エラーバー付き散布図
    plt.errorbar(
        or_df['Odds Ratio'], 
        or_df['Variable'], 
        xerr=[or_df['Odds Ratio'] - or_df['Lower CI'], or_df['Upper CI'] - or_df['Odds Ratio']], 
        fmt='o', 
        color='blue', 
        capsize=5, 
        label='Odds Ratio (95% CI)'
    )
    
    plt.title(f"Odds Ratios of {best_model_name} (Ordinal Logistic)")
    plt.xlabel("Odds Ratio (Exp(Coef))")
    plt.grid(axis='x', linestyle='--', alpha=0.5)
    plt.legend()
    plt.tight_layout()
    plt.savefig(OUTPUT_IMAGE_NAME, dpi=300)
    plt.show()